In [47]:
from gurobipy import *
import networkx as nx
import math
import re
import pandas as pd

In [48]:
def data_load(fileName):
    listSet = []
    list_temp = []
    start_flag = False
    with open(fileName, "r") as file:
        for line in file:
            if line.strip() == "":
                if list_temp:
                    listSet.append(list_temp.copy())
                    # print(list_temp)
                    list_temp.clear()
                start_flag = True;
                continue
            if start_flag == True:
                if line.find(":") > -1:
                    continue
                else:
                    list_temp.append(line.split())

    df_courses = pd.DataFrame(listSet[0], columns=['courseID','teacher','countOfLectures','minWorkingDays','countOfStudents'])
    df_courses = df_courses.set_index('courseID', drop=True)
    df_rooms = pd.DataFrame(listSet[1], columns=['roomID', 'capacity'])
    df_rooms = df_rooms.set_index('roomID', drop=True)
    dict_curricula = {}
    df_unavailability_constraints = pd.DataFrame(listSet[3], columns=['courseID','day','dayPeriod'])
    df_unavailability_constraints = df_unavailability_constraints.set_index('courseID', drop=True)

    dict_temp = {}
    for item in listSet[2]:
        for i in range(len(item)):
            if i == 1:
                dict_temp['count'] = item[i]
            if i == 2:
                dict_temp['members'] = item[i:]
        dict_curricula[item[0]] = dict_temp.copy()
        dict_temp.clear()

    return df_courses, df_rooms, dict_curricula, df_unavailability_constraints

In [49]:
df_courses, df_rooms, dict_curricula, df_unavailability_constraints = data_load('comp02.ctt')

In [51]:
dict_curricula

{'q000': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0157']},
 'q001': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0155']},
 'q002': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0156']},
 'q003': {'count': '3', 'members': ['c0211', 'c0213', 'c0238']},
 'q004': {'count': '3', 'members': ['c0230', 'c0023', 'c0217']},
 'q005': {'count': '4', 'members': ['c0007', 'c0009', 'c0230', 'c0023']},
 'q006': {'count': '4', 'members': ['c0019', 'c0310', 'c0088', 'c0080']},
 'q007': {'count': '4', 'members': ['c0015', 'c0019', 'c0020', 'c0368']},
 'q009': {'count': '4', 'members': ['c0217', 'c0219', 'c0023', 'c0230']},
 'q010': {'count': '4', 'members': ['c0310', 'c0633', 'c0302', 'c0088']},
 'q011': {'count': '3', 'members': ['c0310', 'c0633', 'c0298']},
 'q012': {'count': '3', 'members': ['c0044', 'c0056', 'c0069']},
 'q013': {'count': '3', 'members': ['c0266', 'c0054', 'c0058']},
 'q014': {'count': '4', 'members': ['c0323', 'c0061', 'c0327', 'c0066']},
 'q015': 

In [52]:
df_courses

,teacher,countOfLectures,minWorkingDays,countOfStudents
courseID,,,,
c0131,t000,3,2,150
c0211,t001,3,3,147
c0095,t002,3,3,89
c0152,t003,3,3,150
c0103,t004,3,3,114
...,...,...,...,...
c0337,t066,3,3,64
c0307,t067,5,4,23
c0633,t068,5,4,214


In [53]:
df_rooms

,capacity
roomID,
36,42
37,42
38,48
31,50
27,90
B,216
D,216
E,216
F,216


In [54]:
df_unavailability_constraints

,day,dayPeriod
courseID,,
c0211,0,2
c0211,0,3
c0211,0,4
c0211,1,2
c0211,1,3
...,...,...
c0337,4,3
c0337,4,4
c0368,0,0


In [55]:
dict_curricula

{'q000': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0157']},
 'q001': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0155']},
 'q002': {'count': '4', 'members': ['c0129', 'c0131', 'c0152', 'c0156']},
 'q003': {'count': '3', 'members': ['c0211', 'c0213', 'c0238']},
 'q004': {'count': '3', 'members': ['c0230', 'c0023', 'c0217']},
 'q005': {'count': '4', 'members': ['c0007', 'c0009', 'c0230', 'c0023']},
 'q006': {'count': '4', 'members': ['c0019', 'c0310', 'c0088', 'c0080']},
 'q007': {'count': '4', 'members': ['c0015', 'c0019', 'c0020', 'c0368']},
 'q009': {'count': '4', 'members': ['c0217', 'c0219', 'c0023', 'c0230']},
 'q010': {'count': '4', 'members': ['c0310', 'c0633', 'c0302', 'c0088']},
 'q011': {'count': '3', 'members': ['c0310', 'c0633', 'c0298']},
 'q012': {'count': '3', 'members': ['c0044', 'c0056', 'c0069']},
 'q013': {'count': '3', 'members': ['c0266', 'c0054', 'c0058']},
 'q014': {'count': '4', 'members': ['c0323', 'c0061', 'c0327', 'c0066']},
 'q015': 